In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Datasets

/content/drive/.shortcut-targets-by-id/1Xvmy4JPQNukFURFjRgJQjgR3wNkeCSSK/Datasets


In [ ]:
import pandas as pd
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torchaudio

processor = WhisperProcessor.from_pretrained("openai/whisper-medium.en")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium.en")
model = model.to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Datasets/libristutter/train.csv")

In [ ]:
data

,transcript,audio
0,my clothes wait upon me like a suit of armor s...,audio/4853-27670-0002.flac
1,should not in this be equal to the most stupid...,audio/311-124404-0111.flac
2,but there was nothing funny about it now when ...,audio/5561-41615-0045.flac
3,say the prayer of Santa Apolonia that is if yo...,audio/2989-138028-0039.flac
4,if host of living frogs have come here from so...,audio/3240-131231-0022.flac
...,...,...
3125,reply to Emily. Request would have proved to m...,audio/2007-132570-0045.flac
3126,and in this Garden the youngest princess used ...,audio/2092-145709-0002.flac
3127,I've always been thinking I need better than t...,audio/1963-142393-0030.flac
3128,Andy recoverable moment at the dazzling inters...,audio/5393-19219-0011.flac


In [ ]:
from tqdm import tqdm
import torchaudio
import torch

responses = []

for _, row in tqdm(data.iterrows(), total=len(data)):
    audio_file = "/content/drive/MyDrive/Datasets/libristutter/" + row["audio"]

    # Load the audio file
    waveform, sr = torchaudio.load(audio_file)

    # Resample if the sampling rate is not 16000
    if sr != 16000:
        waveform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)(waveform)
        sr = 16000

    # Ensure waveform is mono (1 channel)
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)


    # Ensure waveform is in the correct shape for the processor
    waveform = waveform.squeeze()  # Remove any singleton dimensions

    # Process input features
    input_features = processor(
      waveform, sampling_rate=sr, return_tensors="pt"
    ).input_features

    # Check input feature shape

    # Move to GPU if available
    if torch.cuda.is_available():
        input_features = input_features.cuda()

    # Generate predictions
    predicted_ids = model.generate(input_features, num_beams=10, num_return_sequences=10)

    # Decode predictions to transcription
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

    # Print transcription
    responses.append({
        "ground_truth": row["transcript"],
        "reference": transcription,
        "path" : row["audio"],
    })

    pd.DataFrame(responses).to_csv("/content/drive/MyDrive/Datasets/libristutter/outputs/whisper-medium-inference-train.csv", index=False)

  0%|          | 0/3130 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
100%|██████████| 3130/3130 [1:58:54<00:00,  2.28s/it]
